<a href="https://colab.research.google.com/github/Theanh130124/MC_Plagiarism-Detector/blob/master/Plagiarism_Detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**IMPORT MODULES**





In [ ]:
import nltk #Xử lý NLP
nltk.download('popular')
import pandas as pd #Làm việc với df
import string #Chuẩn string
from nltk.corpus import stopwords #Stopword -> engs có thể bỏ the an ....
from sklearn.linear_model import LogisticRegression #Mô hình tuyến tình (để phân loại nhị phân (Có , không))
from sklearn.model_selection import train_test_split #Chia train và test
from sklearn.metrics import accuracy_score, classification_report,confusion_matrix #tính accuracy (độ chính xác) hoặc F1-score... , ma trận sai lầm
from sklearn.feature_extraction.text import TfidfVectorizer  #Chuyển đoạn văn bản thành ma trận tần suất xh



[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/names.zip.
[nltk_data]    | Downloading package shakespeare to /root/nltk_data...
[nlt

LOAD DATASET (Note dataset này chưa LLMs)

In [ ]:
data = pd.read_csv("/content/dataset.csv")
data

,Unnamed: 0,source_text,plagiarized_text,label
0,0,Researchers have discovered a new species of b...,Scientists have found a previously unknown but...,1
1,1,The moon orbits the Earth in approximately 27....,Our natural satellite takes around 27.3 days t...,1
2,2,Water is composed of two hydrogen atoms and on...,H2O consists of 2 hydrogen atoms and 1 oxygen ...,1
3,3,The history of Rome dates back to 753 BC.,Rome has a long history that can be traced bac...,1
4,4,Pluto was once considered the ninth planet in ...,"In the past, Pluto was classified as the ninth...",1
...,...,...,...,...
365,397,Playing musical instruments enhances creativity.,Creativity is enhanced by playing musical inst...,0
366,398,Studying history helps in understanding the pr...,Understanding the present is aided by studying...,0
367,399,Listening to classical music can improve focus.,Focus is improved by listening to classical mu...,0
368,400,Practicing yoga enhances physical flexibility.,Physical flexibility is enhanced by practicing...,0


In [ ]:
data['label'].value_counts() #Đây là bước ktra căn bằng mẫu (nếu không cần bằng có thể tăng bên ít , giảm bên nhiều để că bằng)

,count
label,
0,187
1,183


In [ ]:
data.shape #nó có 370 dòng và 4 cột

(370, 4)

Clean Text

In [ ]:
def proprocess_text(text):
  #remove punctation
  text = text.translate(str.maketrans("","",string.punctuation))
  #Convert to lowercase
  text = text.lower()
  #Remove stopwords (the, an, is, .. S , giới từ)
  stop_words = set(stopwords.words('english'))
  #lặp word in text.split() -> cắt ra khoảng , nếu không trong stopw thì giữ và ghép lại bằng join thành text
  text = " ".join(word for word in text.split() if word not in stop_words)
  return text


#string.punctuation là một chuỗi chứa tất cả dấu câu: '!"#$%&\'()*+,-./:;<=>?@[\\]^_{|}~'`

# str.maketrans(x, y, z)
# x: chuỗi các ký tự gốc
# y: chuỗi các ký tự thay thế tương ứng
# z: chuỗi các ký tự sẽ bị xoá

#  Ví dụ đơn giản:
# table = str.maketrans("abc", "123")
# "abcabc".translate(table)   # → '123123'

#Ex:
proprocess_text("This is my $%^$^? text to use for dummt text ")

'text use dummt text'

In [ ]:
data['source_text']=data['source_text'].apply(proprocess_text) #Làm sạch cột source_text
data['plagiarized_text'] = data['plagiarized_text'].apply(proprocess_text) #Làm sạch cột plagiarized_text
data
#Ta thấy index 0 -> mất have ...(trong stops_word)

,Unnamed: 0,source_text,plagiarized_text,label
0,0,researchers discovered new species butterfly a...,scientists found previously unknown butterfly ...,1
1,1,moon orbits earth approximately 273 days,natural satellite takes around 273 days comple...,1
2,2,water composed two hydrogen atoms one oxygen atom,h2o consists 2 hydrogen atoms 1 oxygen atom,1
3,3,history rome dates back 753 bc,rome long history traced back 753 bc,1
4,4,pluto considered ninth planet solar system,past pluto classified ninth planet suns planet...,1
...,...,...,...,...
365,397,playing musical instruments enhances creativity,creativity enhanced playing musical instruments,0
366,398,studying history helps understanding present,understanding present aided studying history,0
367,399,listening classical music improve focus,focus improved listening classical music,0
368,400,practicing yoga enhances physical flexibility,physical flexibility enhanced practicing yoga,0


Vectorzation (Chuyển văn bản thành vector số) để học máy hiểu

In [ ]:
vector = TfidfVectorizer()
#Chuyển đổi qua vector
X = vector.fit_transform(data['source_text']+" "+ data['plagiarized_text'])


In [ ]:
y = data['label']

Train Test Split (Chia Train va test)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)  #train_test_split import o tren  , Chia ra vì để tập test là dự liệu mới (tránh khớp)

Applying logistic regression

In [ ]:
model = LogisticRegression() #Đây là bước khởi tạo mô hình học máy, cụ thể là Logistic Regression – một mô hình phân loại phổ biến dùng để dự đoán nhãn (label) nhị phân hoặc đa lớp.
model.fit(X_train, y_train) # Đây là bước huấn luyện (training) mô hình với dữ liệu đã chia:
y_pred = model.predict(X_test) #Ta se ktra do chinh xac mo hinh du tren dL test


print("accuracy",accuracy_score(y_test, y_pred)) #accuracy 0.8243243243243243
print("Classification",classification_report(y_test, y_pred)) #Tong quan
print("confusion",confusion_matrix(y_test, y_pred)) #Matran sai lam

# X_train: đặc trưng đầu vào (features), ví dụ chiều cao, cân nặng,...

# y_train: nhãn đầu ra (labels), ví dụ: nam/nữ hoặc có/không bị bệnh.

#  .fit() là cho mô hình học mối quan hệ giữa đầu vào (X_train) và đầu ra (y_train), từ đó học được "đường phân cách" giữa các lớp

accuracy 0.8243243243243243
Classification               precision    recall  f1-score   support

           0       0.79      0.86      0.82        35
           1       0.86      0.79      0.83        39

    accuracy                           0.82        74
   macro avg       0.83      0.83      0.82        74
weighted avg       0.83      0.82      0.82        74

confusion [[30  5]
 [ 8 31]]


In [ ]:
# Do chinh xac cho 0 la 0.79 va 1 la 0.86

Random Forest Model

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=100,random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print("accuracy",accuracy_score(y_test, y_pred))
print("Classification",classification_report(y_test, y_pred))
print("confusion",confusion_matrix(y_test, y_pred))

#Ta thay do chinh xac thap hon accuracy 0.7972972972972973

accuracy 0.7972972972972973
Classification               precision    recall  f1-score   support

           0       0.71      0.97      0.82        35
           1       0.96      0.64      0.77        39

    accuracy                           0.80        74
   macro avg       0.83      0.81      0.79        74
weighted avg       0.84      0.80      0.79        74

confusion [[34  1]
 [14 25]]


Naiv Bays Model

In [ ]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
model.fit(X_train, y_train) # Đây là bước huấn luyện (training) mô hình với dữ liệu đã chia:
y_pred = model.predict(X_test) #Ta se ktra do chinh xac mo hinh du tren dL test

print("accuracy",accuracy_score(y_test, y_pred))
print("Classification",classification_report(y_test, y_pred)) #Tong quan
print("confusion",confusion_matrix(y_test, y_pred)) #Matran sai lam


#Mo hinh du doan nay tot

**SVM** Đặc trưng (feature) X	Nhãn (y)
[1, 2]	0
[2, 3]	0
[3, 3]	1
[4, 4]	1

SVM sẽ tìm một đường thẳng chia dữ liệu thành 2 phần, sao cho:

Các điểm lớp 0 nằm một bên.

Các điểm lớp 1 nằm một bên.

Khoảng cách từ đường thẳng đến các điểm gần nhất của 2 lớp là lớn nhất có thể.

In [ ]:
from sklearn.svm import SVC
model = SVC(kernel='linear',random_state=42)
model.fit(X_train, y_train) # Đây là bước huấn luyện (training) mô hình với dữ liệu đã chia:
y_pred = model.predict(X_test) #Ta se ktra do chinh xac mo hinh du tren dL test

print("accuracy",accuracy_score(y_test, y_pred))
print("Classification",classification_report(y_test, y_pred)) #Tong quan
print("confusion",confusion_matrix(y_test, y_pred)) #Matran sai lam

#Do chinh xac cao nhat

accuracy 0.8783783783783784
Classification               precision    recall  f1-score   support

           0       0.86      0.89      0.87        35
           1       0.89      0.87      0.88        39

    accuracy                           0.88        74
   macro avg       0.88      0.88      0.88        74
weighted avg       0.88      0.88      0.88        74

confusion [[31  4]
 [ 5 34]]


Save SVM and Vectorizor

In [ ]:
import pickle #luu dl duoi dang nhi phan
pickle.dump(model, open('model.pkl', 'wb'))
pickle.dump(vector, open('vector.pkl', 'wb'))#can vector (vi may chi hieu DL so (vector))

LOAD MODEL and Vectorizer

In [ ]:
model = pickle.load(open('model.pkl', 'rb'))
vector = pickle.load(open('vector.pkl', 'rb'))

Dection System

In [21]:
def detect(input_text):
  #Chuyen text -> vector
  vectorized_text = vector.transform([input_text])

  #Thuc hien model
  res = model.predict(vectorized_text)
  return "Day la dao van" if res[0] == 1 else  "Khong phai dao van"

Test

In [23]:
input_text ="pluto considered ninth planet solar system"

detect(input_text)

'Day la dao van'